In [4]:
import os
from IPython.display import Markdown as md
from dotenv import load_dotenv
from mistralai import Mistral
from llama_cloud_services import LlamaParse
import base64
import nest_asyncio
import asyncio


nest_asyncio.apply()
load_dotenv()

link_SEH_form = "../data/cash/seh/LPF825_fr.pdf"
link_SEH_help_hard = "../data/cash/seh/Fiche-aides-financieres-moins-de-18-ans_mise-a-jour-le-18-dec.-2024.pdf"
link_SEH_help_2 = "../data/cash/seh/SEH_Nouvelle_Version.pdf"



In [22]:
# Mistral
class Mistral_ocr:
    def __init__(self,document_url):
        self.client = Mistral(api_key=os.environ["MISTRAL_API_KEY"])
        self.document_url = document_url
        self.ocr_response = self.get_ocr_response()
    def encode_pdf(self,pdf_path):
        """Encode the pdf to base64."""
        try:
            with open(pdf_path, "rb") as pdf_file:
                return base64.b64encode(pdf_file.read()).decode('utf-8')
            
        except FileNotFoundError:
            print(f"Error: The file {pdf_path} was not found.")
            return None
        except Exception as e:  # Added general exception handling
            print(f"Error: {e}")
            return None
    def get_ocr_response(self):
        print(self.document_url)
        base64_pdf = self.encode_pdf(self.document_url)
        ocr_response_form = self.client.ocr.process(
                model="mistral-ocr-latest",
                document={
                    "type": "document_url",
                    "document_url": f"data:application/pdf;base64,{base64_pdf}" 
                },
                include_image_base64=True
            )
        return ocr_response_form

    def get_markdown_from_page(self, page_index):
        return self.ocr_response.pages[page_index].markdown

In [25]:
Mistra_form = Mistral_ocr(link_SEH_form)
Mistral_help_hard = Mistral_ocr(link_SEH_help_hard)
Mistral_help_2 = Mistral_ocr(link_SEH_help_2)



../data/cash/seh/LPF825_fr.pdf
yeaa
../data/cash/seh/Fiche-aides-financieres-moins-de-18-ans_mise-a-jour-le-18-dec.-2024.pdf
yeaa
../data/cash/seh/SEH_Nouvelle_Version.pdf
yeaa


In [26]:
md(Mistral_help_2.get_markdown_from_page(3))

![img-7.jpeg](img-7.jpeg)

Des fiches pratiques sur l'autisme pour vous accompagner au quotidien

## Nutrition

Prise des repas, utilisation des accessoires pour boire et manger

- Mon enfant ne mange pas seul.
- Mon enfant ne peut pas choisir son repas.
- Il ne sait pas faire réchauffer son plat.
- Je dois couper tous ses aliments.
- Je dois constamment l'encourager à manger.
- Mon enfant n'arrive pas à tenir ses ustensiles.
- Mon enfant a des hypersensibilités gustatives, les repas lui apportent beaucoup d'anxiété.
- Les repas sont très longs, car mon enfant mange lentement et doit être constamment supervisé.
- Mon enfant se fait vomir au moindre problème lié à son alimentation.
- Mon enfant se désorganise et l'intensité de ces crises est élevée s'il y a un élément qu'il n'aime pas dans son assiette.
- Mon enfant peut vomir pendant les repas (hypersensibilité olfactive/gustative). La texture de ses aliments doit respecter ses préférences pour parvenir à ce qu'il s'alimente.
- Mon enfant ne reste pas assis durant le repas, on doit le superviser pour s'assurer qu'il mange.
- La motricité fine de mon enfant fait en sorte que l'utilisation des ustensiles est vraiment difficile, il faut l'aider à chaque repas.
- À cause de sa médication, mon enfant s'alimente très peu, je dois le superviser/motiver pour éviter la perte de poids.

Voici des exemples de phrases à utiliser dans votre lettre afin de préciser les caractéristiques de votre enfant.

## Soins personnels

Hygiène corporelle, hygiène excrétrice (urine et selles), habillage, prise de médicaments.

- Mon enfant n'a pas fait l'apprentissage de la propreté.
- Mon enfant ne prend pas sa douche ou son bain seul.
- Je dois constamment superviser mon enfant dans son bain.
- Mon enfant ne prend pas ses médicaments seuls.
- Mon enfant ne choisit pas lui-même ses vêtements.
- Il ne s'habille pas seul.
- Mon enfant s'habille seul, mais je dois lui donner les étapes verbalement ou avec des pictogrammes une par une.
- Mon enfant n'arrive pas à attacher ses souliers seuls, ni son manteau.
- Mon enfant ne sait pas s'habiller correctement selon les saisons, il ne sent ni la chaleur, ni le froid et se met à risque de coup de chaleur ou d'hypothermie.
- Si mon enfant n'aime pas ses vêtements (hypersensibilité), il peut se désorganiser et refuser d'aller à l'école ou il peut se dévêtir en public.
- S'il n'aime pas ses vêtements, l'intensité de ces crises n'est pas normale pour sa période développementale.
- Mon enfant ne peut pas aller à la toilette seul à l'école.
- Mon enfant refuse d'utiliser les toilettes à l'école.
- Mon enfant porte des bobettes, mais c'est l'adulte qui doit lui faire penser d'aller au toilette (routine).
- Mon enfant est propre pour les pipis, mais pas pour les selles. Il faut utiliser du laxatif, des séquences visuelles et parfois des lavements car il ne sent pas bien son corps et ses envies d'aller à la selle.
- Je dois brosser les dents de mon enfant.
- Il refuse d'utiliser une brosse à dents à cause d'hypersensibilité.

In [5]:
class LlamaParse_ocr:
    def __init__(self,document_url):
        self.parser = LlamaParse(
            api_key=os.environ["LLAMA_CLOUD_API_KEY"],
        verbose=True,
        language="fr",       # optionally define a language, default=en
        premium_mode=True
        )
        self.document_url = document_url
        self.ocr_response = self.get_ocr_response()     

    def get_ocr_response(self):
        try:
            loop = asyncio.get_running_loop()
            # Si on est déjà dans une boucle, on utilise la version async
            return loop.run_until_complete(self.parser.aparse(self.document_url))
        except RuntimeError:
            # Sinon, on lance une nouvelle boucle
            return asyncio.run(self.parser.aparse(self.document_url))

    def get_markdown_from_page(self, page_index):
        markdown_documents = self.ocr_response.get_markdown_documents(split_by_page=True)
        return markdown_documents[page_index]


In [6]:
LlamaParse_help_hard = LlamaParse_ocr(link_SEH_help_hard)
result = LlamaParse_help_hard.get_ocr_response()


Started parsing the file under job_id cb366452-78b4-450d-aa9d-f3f92ef03df2
Started parsing the file under job_id 5c848ba3-f57a-451e-b6a5-5f837c57472f


In [8]:
print(result)

pages=[Page(page=1, text='        PARLE-MOI DE TSA\n\n          LES AIdES\n       FINANCIERES\n       pour les moins de 18 ans\n\nÉclairage sur les aides financières et programmes\n sociaux disponibles pour les enfants autistes et\n   leurs parents ou proches aidants aux niveaux\n         fédéral, provincial et municipal.\n                42 utisme           02\n            www.autisme-montreal.com', md='# PARLE-MOI DE TSA\n\n## LES AIDES FINANCIÈRES\n### pour les moins de 18 ans\n\nÉclairage sur les aides financières et programmes sociaux disponibles pour les enfants autistes et leurs parents ou proches aidants aux niveaux fédéral, provincial et municipal.\n\nAutisme Montréal\n\nwww.autisme-montreal.com', images=[ImageItem(name='img_p0_1.png', height=1141.0, width=1416.0, x=-12.48416059982662, y=762.138492244229, original_width=1416, original_height=1141, type=None), ImageItem(name='img_p0_2.png', height=2145.0, width=2552.0, x=-0.23999999, y=117.60001509999984, original_width=2552, o